In [3]:
pip install gradio pandas scikit-learn joblib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import gradio as gr
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Load the model
model = joblib.load('predictive_maintenance_model.pkl')

# Define a function to make predictions
def predict_maintenance(file):
    # Load the new data from the uploaded file
    test_data = pd.read_csv(file.name)

    # Drop the 'Timestamp' column if it exists
    #if 'Timestamp' in test_data.columns:
        #post_data = test_data.drop(columns=['Timestamp','Machine ID'])
        
    post_data = test_data.drop(columns=['Timestamp','Machine ID'])    
    
    # Scale numerical features (excluding categorical features)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(post_data.select_dtypes(include=['float', 'int']))
    X_scaled = pd.DataFrame(X_scaled, columns=post_data.select_dtypes(include=['float', 'int']).columns)

    # Combine scaled and unscaled features
    X = pd.concat([X_scaled, post_data.select_dtypes(include=['object'])], axis=1)

    # Ensure that the new data has the same features as the training data
    # The test data should only include feature columns, not the target column

    # Predict maintenance requirement for each row in the new data
    predictions = model.predict(X)

    # Add predictions to the new data DataFrame
    test_data['maintenance_prediction'] = predictions

    # Save the results to a new CSV file with predictions
    output_file = 'test_data_with_predictions.csv'
    test_data.to_csv(output_file, index=False)

    return output_file

# Define the Gradio interface
iface = gr.Interface(
    fn=predict_maintenance,
    inputs=gr.File(label="Upload CSV file"),
    outputs=gr.File(label="Download CSV file with predictions"),
    title="Predictive Maintenance",
    description="Upload a CSV file of sensor data to predict maintenance requirements. The output will be a CSV file with the predictions included."
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
